# First results

This notebook is a first look at the CM26 data. It can serve as a starting point for others to access the data.
To get your token json file, contact me on keybase!

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import json
import gcsfs
from dask.diagnostics import ProgressBar

# 👇 replace with your key 
with open('/home/jovyan/keys/pangeo-forge-ocean-transport-4967-347e2048c5a1.json') as token_file:
    token = json.load(token_file)
fs = gcsfs.GCSFileSystem(token=token)
subfolder_full = 'ocean-transport-group/scale-aware-air-sea/outputs/temp/'
subfolder_final = 'ocean-transport-group/scale-aware-air-sea/outputs/'

In [3]:
# reload the final store (needs the first cell to be executed, but should otherwise work independently)
# path = f'{subfolder_final}CM26_final_output.zarr'# 5 year version
path = f'{subfolder_final}CM26_final_output_full_time.zarr' # full time (20yrs)
mapper = fs.get_mapper(path)
ds_plot = xr.open_dataset(mapper, engine='zarr', consolidated=True, use_cftime=True, chunks='auto')

ds_plot = ds_plot.sel(yt_ocean=slice(-60,60))
ds_plot

<xarray.Dataset>
Dimensions:           (yt_ocean: 75, xt_ocean: 180, time: 7305)
Coordinates:
    area              (yt_ocean, xt_ocean) float32 dask.array<chunksize=(75, 180), meta=np.ndarray>
    area_t            (yt_ocean, xt_ocean) float32 dask.array<chunksize=(75, 180), meta=np.ndarray>
    dxt               (yt_ocean, xt_ocean) float64 dask.array<chunksize=(75, 180), meta=np.ndarray>
    dyt               (yt_ocean, xt_ocean) float64 dask.array<chunksize=(75, 180), meta=np.ndarray>
    geolat_t          (yt_ocean, xt_ocean) float32 dask.array<chunksize=(75, 180), meta=np.ndarray>
    geolon_t          (yt_ocean, xt_ocean) float32 dask.array<chunksize=(75, 180), meta=np.ndarray>
    ht                (yt_ocean, xt_ocean) float32 dask.array<chunksize=(75, 180), meta=np.ndarray>
    kmt               (yt_ocean, xt_ocean) float32 dask.array<chunksize=(75, 180), meta=np.ndarray>
  * time              (time) object 0181-01-01 12:00:00 ... 0200-12-31 12:00:00
    wet               (yt_ocean, xt_ocean) float64 dask.array<chunksize=(75, 180), meta=np.ndarray>
  * xt_ocean          (xt_ocean) float64 -279.0 -277.0 -275.0 ... 75.0 77.0 79.0
  * yt_ocean          (yt_ocean) float64 -59.01 -57.96 -56.88 ... 58.49 59.52
Data variables: (12/16)
    evap              (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(180, 75, 1380), meta=np.ndarray>
    evap_large_scale  (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(180, 75, 1380), meta=np.ndarray>
    q_ref             (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(180, 75, 1380), meta=np.ndarray>
    qh                (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(180, 75, 1380), meta=np.ndarray>
    qh_large_scale    (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(180, 75, 1380), meta=np.ndarray>
    ql                (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(180, 75, 1380), meta=np.ndarray>
    ...                ...
    taux              (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(180, 75, 1380), meta=np.ndarray>
    taux_large_scale  (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(180, 75, 1380), meta=np.ndarray>
    tauy              (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(180, 75, 1380), meta=np.ndarray>
    tauy_large_scale  (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(180, 75, 1380), meta=np.ndarray>
    u_ref             (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(180, 75, 1380), meta=np.ndarray>
    v_ref             (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(180, 75, 1380), meta=np.ndarray>

In [ ]:
with ProgressBar():
    ds_mean = ds_plot.mean('time').load()

[                                        ] | 0% Completed | 11.4s

In [ ]:
for var in ['ql', 'qh', 'evap', 'taux', 'tauy']:
    with ProgressBar():
        full = ds_mean[var]
        large_scale = ds_mean[var+'_large_scale']
        small_scale = full-large_scale

        # not quite sure if we should look at the mean of the difference or the difference of the mean over time...
        plot_kwargs = {'y':'yt_ocean', 'robust':True, 'center':0}
        plt.figure(figsize=[30,4])
        plt.subplot(1,5,1)
        full.plot(**plot_kwargs)
        plt.title('Full output coarsened')
        plt.subplot(1,5,2)
        large_scale.plot(**plot_kwargs)
        plt.title('Output from coarsened input')
        plt.subplot(1,5,3)
        small_scale.plot(**plot_kwargs)
        plt.title('Small Scale Absolute')
        plt.subplot(1,5,4)
        (small_scale/full*100).plot(vmax=10, **plot_kwargs)
        plt.title('Small Scale Relative')
        
        plt.subplot(1,5,5)
        (small_scale/full*100).plot.hist(bins=np.linspace(0,20,10))
        print(f"{var} median: {(small_scale/full*100).quantile(0.5).data}")
        plt.yscale('log')
        plt.title('Small Scale Relative Hist')
        plt.show()

In [ ]:
with ProgressBar():
    ds_seasonal = ds_plot.groupby('time.season').mean('time').load()
ds_seasonal = ds_seasonal.sortby(xr.DataArray(['DJF','MAM','JJA', 'SON'], dims=['season']))

In [ ]:
# Check out the seasonal cycle
for var in ['ql', 'qh', 'evap', 'taux', 'tauy']:
    with ProgressBar():
        full = ds_seasonal[var]
        large_scale = ds_seasonal[var+'_large_scale']
        small_scale = full-large_scale

        # not quite sure if we should look at the mean of the difference or the difference of the mean over time...
        plot_kwargs = {'y':'yt_ocean', 'robust':True, 'center':0, 'col':'season'}
        
        plt.figure(figsize=[30,4])
        full.plot(**plot_kwargs)
        plt.suptitle('Full output coarsened', y=0.99)
        
        plt.figure(figsize=[30,4])
        large_scale.plot(**plot_kwargs)
        plt.suptitle('Output from coarsened input', y=0.99)
        
        plt.figure(figsize=[30,4])
        small_scale.plot(**plot_kwargs)
        plt.suptitle('Small Scale Absolute', y=0.99)
        
        plt.figure(figsize=[30,4])
        (small_scale/full*100).plot(vmax=10, **plot_kwargs)
        plt.suptitle('Small Scale Relative', y=0.99)
        plt.show()